# Day 1

https://adventofcode.com/2021/day/1

In [1]:
with open("./input/1") as file:
    depths = [int(line) for line in file]

## Part 1

In [2]:
sum(int(d2 > d1) for d1, d2 in zip(depths[:-1], depths[1:]))

1477

## Part 2

In [3]:
sum(int(d4 > d1) for d1, d4 in zip(depths[:-3], depths[3:]))

1523

# Day 2

https://adventofcode.com/2021/day/2

In [4]:
from numpy import array  # I'm lazy

In [5]:
def parse_instruction(instruction):
    direction, _distance = instruction.split()
    distance = int(_distance)
    if direction == "forward":
        return (distance, 0)
    return (0, (1 if direction == "down" else -1) * distance)

In [6]:
with open("./input/2") as file:
    instructions = array([parse_instruction(line) for line in file])

## Part 1

In [7]:
forward_distance, depth = instructions.sum(axis=0)

In [8]:
forward_distance * depth

1936494

## Part 2

In [9]:
forward_instr, aim_instr = instructions.T

In [10]:
sum(forward_instr) * (forward_instr * aim_instr.cumsum()).sum()

1997106066

# Day 3

https://adventofcode.com/2021/day/3

In [11]:
from numpy import array

In [12]:
with open("./input/3") as file:
    bit_list = array([[int(bit) for bit in line[:-1]] for line in file])

In [13]:
def bits_to_int(bits):
    return int("".join([str(bit) for bit in bits]), base=2)

## Part 1

In [14]:
gamma = bits_to_int(["1" if avg > 0.5 else "0" for avg in bit_list.mean(axis=0)])
epsilon = bits_to_int(["0" if avg > 0.5 else "1" for avg in bit_list.mean(axis=0)])

In [15]:
gamma * epsilon

4006064

## Part 2

In [16]:
def _filter_bits(bit_list, position, gas="o2"):
    most_common_bit = int(bit_list[:, position].mean(axis=0) + 0.5)
    if gas == "o2":
        return bit_list[[bits[position] == most_common_bit for bits in bit_list]]
    if gas == "co2":
        return bit_list[[bits[position] == 1 - most_common_bit for bits in bit_list]]
    raise ValueError(f"Expected o2 or co2 gas, got {gas}.")

In [17]:
def reduce_bits(bit_list, gas="o2"):
    current_list = bit_list
    for position in range(bit_list.shape[1]):
        current_list = _filter_bits(current_list, position, gas)
        if len(current_list) == 1:
            return current_list[0]
    raise RuntimeError("Did not find a unique result")

In [18]:
bits_to_int(reduce_bits(bit_list, "o2")) * bits_to_int(reduce_bits(bit_list, "co2"))

5941884

# Day 4

https://adventofcode.com/2021/day/4

In [19]:
from numpy import array

In [20]:
with open("./input/4") as file:
    calls = [int(num) for num in next(file).split(",")]
    _ = next(file)  # second line is empty
    boards = []
    _board = []
    for line in file:
        if line == "\n":
            boards.append(_board)
            _board = []
            continue
        _board.append([int(s) for s in line.split()])

In [21]:
def call_number(board, number):
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == number:
                return i, j

In [22]:
def has_won(board_calls, board_height, board_width):
    full_rows = [[(i, j) for j in range(board_width)] for i in range(board_height)]
    full_cols = [[(i, j) for i in range(board_height)] for j in range(board_width)]
    return (any(all(coord in board_calls for coord in full_row) for full_row in full_rows)
            or any(all(coord in board_calls for coord in full_col) for full_col in full_cols))

## Part 1

In [23]:
def bingo(boards, calls):
    state = {i: [] for i in range(len(boards))}
    for call in calls:
        for i, board in enumerate(boards):
            call_position = call_number(board, call)
            if call_position:
                board_state = state[i]
                board_state.append(call_position)
                if has_won(board_state, len(board), len(board[1])):  # B I N G O
                    unmarked = sum(board[i][j] for i in range(len(board)) for j in range(len(board[i]))
                                   if (i, j) not in board_state)
                    return unmarked * call

In [24]:
bingo(boards, calls)

10680

## Part 2

In [25]:
def losing_bingo(boards, calls):
    state = {i: [] for i in range(len(boards))}
    remaining_boards = set(range(len(boards)))
    for call in calls:
        for i, board in enumerate(boards):
            if i not in remaining_boards:
                continue
            call_position = call_number(board, call)
            if call_position:
                board_state = state[i]
                board_state.append(call_position)
                if has_won(board_state, len(board), len(board[1])):
                    if len(remaining_boards) > 1:
                        remaining_boards.remove(i)
                        continue
                    unmarked = sum(board[i][j] for i in range(len(board)) for j in range(len(board[i]))
                                   if (i, j) not in board_state)
                    return unmarked * call

In [26]:
losing_bingo(boards, calls)

31892

# Day 5

https://adventofcode.com/2021/day/5

In [27]:
from collections import defaultdict

In [28]:
def parse_coord(coord):
    x, y = coord.split(",")
    return int(x), int(y)

In [29]:
with open("./input/5") as file:
    lines = [[parse_coord(start), parse_coord(end)]
             for start, end in [line.split(" -> ") for line in file.read().splitlines()]]

In [30]:
def points_in_line(start, end):
    (start_x, start_y), (end_x, end_y) = sorted([start, end])
    if start_x == end_x:  # vertical
        return [(start_x, y) for y in range(start_y, end_y + 1)]
    if start_y == end_y:  # horizontal
        return [(x, start_y) for x in range(start_x, end_x + 1)]
    if start_y < end_y:  # "upwards" diagonal
        return [(start_x + i, start_y + i) for i in range(end_x - start_x + 1)]
    # "downwards" diagonal
    return [(start_x + i, start_y - i) for i in range(end_x - start_x + 1)]

## Part 1

In [31]:
def fill_grid_no_diagonals():
    grid = defaultdict(lambda: 0)
    for start, end in lines:
        if start[0] != end[0] and start[1] != end[1]:
            continue
        for point in points_in_line(start, end):
            grid[point] += 1
    return grid

In [32]:
sum(1 for point, overlap in fill_grid_no_diagonals().items() if overlap > 1)

4993

## Part 2

In [33]:
def fill_grid():
    grid = defaultdict(lambda: 0)
    for start, end in lines:
        for point in points_in_line(start, end):
            grid[point] += 1
    return grid

In [34]:
sum(1 for point, overlap in fill_grid().items() if overlap > 1)

21101